## Semillero de Programación en Ciencias Sociales (Sesión 5)
### Felipe Botero, Laura Merchán y Wilson Forero

In [1]:
#conda install -c conda-forge altair vega_datasets

In [2]:
#!pip install altair vega_datasets

#### El objetivo de esta clase es descargar automáticamente todos los fallos que la Corte Constitucional ha producido sobre un tema y realizar una gráfica dónde se evidencie cuántos fallos ha producido sobre este tema cada año

<img src="visualization.png">

### Paquetes que vamos a utilizar:

**[1.OS](https://docs.python.org/3/library/os.html):** Permite interactuar con el sistema operativo. Lo usaremos para manipular la estructura de directorios (Crear o eliminar directorios o archivos).

**[2.requests:](https://pypi.org/project/requests/)** Permite automatizar la navegación en internet (Entrar  y moverse entre páginas).

**[3.urlib.parse:](https://docs.python.org/3/library/urllib.parse.html#module-urllib.parse)** Permite manipular urls (Fragmentarlas  o unirlas).

**[4.BeautifulSoup:](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)** La estrella de hoy. Es una librería especializada en extraer información de la web.

**[5.RegEx:](https://www.w3schools.com/python/python_regex.asp)** Permite trabajar con [expresiones regulares](https://en.wikipedia.org/wiki/Regular_expression).Las expresiones regulares son una serie de convenciones para simplificar la búsqueda de strings que siguen algunos patrones.

**[6. Altair](https://altair-viz.github.io/gallery/)** Una librería para visualizar fácilmente datos.

**[7.Pandas](https://pandas.pydata.org/)** La librería insignia de  python para manipular datos.



In [3]:
import os
import requests
import urllib.parse
from bs4 import BeautifulSoup
import re
import altair as alt
import pandas as pd

Lo primero que hacemos es crear una carpeta para guardar las sentencias.

In [4]:
#If there is no such folder, the script will create one automatically
folder_location = './sentencias_aborto'
if not os.path.exists(folder_location):
    os.mkdir(folder_location)

Revisemos la página de la Corte (https://www.corteconstitucional.gov.co/relatoria/)
Con *urllib.parse* podemos ver las partes de la url.

In [5]:
url= "https://www.corteconstitucional.gov.co/relatoria/query.aspx?pg=0&buscar=aborto&anio=/relatoria/&total=422"
urllib.parse.urlparse(url)

ParseResult(scheme='https', netloc='www.corteconstitucional.gov.co', path='/relatoria/query.aspx', params='', query='pg=0&buscar=aborto&anio=/relatoria/&total=422', fragment='')

- Con *requests* visitamos la página de internet almacenada en *url* y la almacenamos en la variable *response*.
- Con *BeautifulSoup* organizamos la información en una estructura jerárquica de  acuerdo a la estructura del código *html* de la página.
- Con *soup.prettify* podemos ver esta estructura. Pueden visitar este [link](https://www.w3schools.com/TAGS/tag_a.asp) comprender mejor el código html.

In [6]:
response = requests.get(url)
soup= BeautifulSoup(response.text, "html.parser")

In [7]:
print(soup.prettify()[15500:16900])

<hr/>
     </div>
     <div align="justify" class="grow">
      <h4>
       <a href="/Relatoria/autos/2019/A333-19.htm">
        A333-19
       </a>
       <small>
        <i>
         Tamaño 60514 bytes
        </i>
       </small>
      </h4>
      <p>
       <b>
        3.
       </b>
       A333-19              
          Auto  333/19
 Referencia:  Expedientes D-13225, demanda de inconstitucionalidad contra los artículos 90,  91 y 93 del Código Civil; y D-13255, demanda de inconstitucionalidad contra el  artículo 122 del Código Penal Asunto: Incidentes  de Recusación formulados contra el Magistrado Alejandro Linares Cantillo por la  abogada Natalia Bernal Cano Magistrado  Ponente:  ANTONIO  JOSÉ LIZARAZO OCAMPO Bogotá D. C.,  diecinueve (19) de junio de dos mil  diecinueve (2019) La Sala Plena de la Corte Constitucional, en  cumplimiento de sus atribuciones constitucionales y legales, procede a resolver  el asunto de la referencia.  I.   ANTECEDENTES A través de sendos escritos rad

Como podemos ver, cada decisión de la corte es un elemento de clase *grow*. Dentro de esta clase se encuentra el hipervínculo a la decisión demarcado por la etiqueta **a** y la url se marca con la etiqueta **href**. En general,los hipervínculos de las páginas web usan estas etiquetas. 

Para extraer los hipervínculos vamos a buscar todos los elementos de clase *grow* dentro del código de la página, luego buscamos los hipervínculos (a) en cada decisión y finalmente su link (href). 

In [8]:
#Retorna una lista con los elementos de clase grow
decisiones= soup.find_all(class_="grow")

In [9]:
print(type(decisiones))
print(decisiones[0])

<class 'bs4.element.ResultSet'>
<div align="justify" class="grow"><h4><a href="/Relatoria/autos/2019/A574-19.htm"> A574-19</a><small> <i>Tamaño 135095 bytes  </i></small></h4><p><b>1.</b> A574-19              
          Auto 574/19 Referencia:   Expediente   T-7.568.177 Acción  de tutela instaurada por Julián Daza Malo y José Luis Chimoquero Gil contra la Alcaldía  de Valledupar. Asunto:  Medida provisional de protección. Procedencia:  Juzgado Primero Penal Municipal de Valledupar con Funciones de Control de  Garantías. Magistrada  sustanciadora: GLORIA  STELLA ORTIZ DELGADO Bogotá, D. C., veintidós (22) de octubre  de dos mil dieciocho (2019) La Sala Sexta de Revisión de la  Corte Constitucional, conformada por el magistrado José Fernando Reyes Cuartas  y por las magistradas Cristina Pardo Schlesinger y Gloria Stella Ortiz Delgado,  quien la preside, en ejercicio de sus competencias constitucionales y legales,  en especial las contenidas en el artículo 7° del Decreto 2591 de 1991 [1] 

Como cada decisión es un elemento de la lista decisiones, debemos hacer un for loop para extraer los links de cada decisión

In [10]:
links=[]
for decision in decisiones:
    hipervinculo=decision.find("a")
    links.append(hipervinculo.get("href"))

In [11]:
links[0:10]

['/Relatoria/autos/2019/A574-19.htm',
 '/Relatoria/autos/2019/A558-19.htm',
 '/Relatoria/autos/2019/A333-19.htm',
 '/Relatoria/autos/2019/A140-19.htm',
 '/Relatoria/autos/2018/A616-18.htm',
 '/Relatoria/autos/2018/A557-18.htm',
 '/Relatoria/autos/2018/A547-18.htm',
 '/Relatoria/autos/2017/A642-17.htm',
 '/Relatoria/autos/2017/A641-17.htm',
 '/Relatoria/autos/2017/A448-17.htm']

La url completa a la decisión será el link de la página de origen concatenada con el link que extrajimos. Entonces debemos concatenar estos dos strings. Este procedimiento debemos hacerlo para cada *(for)* link.

In [12]:
links=[]
for decision in decisiones:
    hipervinculo=decision.find("a")
    link=hipervinculo.get("href")
    link_completo=urllib.parse.urljoin(url,link)
    links.append(link_completo)

In [13]:
links[0:10]

['https://www.corteconstitucional.gov.co/Relatoria/autos/2019/A574-19.htm',
 'https://www.corteconstitucional.gov.co/Relatoria/autos/2019/A558-19.htm',
 'https://www.corteconstitucional.gov.co/Relatoria/autos/2019/A333-19.htm',
 'https://www.corteconstitucional.gov.co/Relatoria/autos/2019/A140-19.htm',
 'https://www.corteconstitucional.gov.co/Relatoria/autos/2018/A616-18.htm',
 'https://www.corteconstitucional.gov.co/Relatoria/autos/2018/A557-18.htm',
 'https://www.corteconstitucional.gov.co/Relatoria/autos/2018/A547-18.htm',
 'https://www.corteconstitucional.gov.co/Relatoria/autos/2017/A642-17.htm',
 'https://www.corteconstitucional.gov.co/Relatoria/autos/2017/A641-17.htm',
 'https://www.corteconstitucional.gov.co/Relatoria/autos/2017/A448-17.htm']

Finalmente, también para cada decisión, creamos un archivo con el nombre de la decisión *(filename)* y le decimos que en este archivo escriba *(f.write())* el contenido del hipervínculo

In [14]:
links=[]
for decision in decisiones:
    hipervinculo=decision.find("a")
    link=hipervinculo.get("href")
    link_completo=urllib.parse.urljoin(url,link)
    links.append(link_completo)
    filename=os.path.join(folder_location,link.split('/')[-1])
    with open(filename, 'wb') as f:
        f.write(requests.get(link_completo).content)
    

### ¿Qué falta?

Este script sólo extrae la primera página del buscador.
**Challenge¿Cómo podemos hacer que extraiga la información de las 4 páginas?
Pista:** La sección de *query* de la url tiene un parámetro de página (pg=0). Se debe crear una lista de urls en la que cada elemento sea la url original pero con el parámetro pg= 1,2,3 y 4. Luego se debe iterar el algoritmo desarrollado en clase para cada elemento de esta lista.

In [15]:
urls=[]
for i in range(0,5):
    url= "https://www.corteconstitucional.gov.co/relatoria/query.aspx?pg="+str(i)+"&buscar=aborto&anio=/relatoria/&total=422"
    urls.append(url)
links=[]
for url in urls:
    response = requests.get(url)
    soup= BeautifulSoup(response.text, "html.parser")
    decisiones= soup.find_all(class_="grow")
    for decision in decisiones:
        hipervinculo=decision.find("a")
        link=hipervinculo.get("href")
        link_completo=urllib.parse.urljoin(url,link)
        links.append(link_completo)
        filename=os.path.join(folder_location,link.split('/')[-1])
        with open(filename, 'wb') as f:
            f.write(requests.get(link_completo).content)

In [16]:
years=[]
for link in links:
    try:
        year=re.findall(r"[0-9][0-9].htm",link)[0]
        year=year[0:2]
        years.append(year)
    except:
        next
print(years)

['19', '19', '19', '19', '18', '18', '18', '17', '17', '17', '17', '16', '16', '16', '16', '15', '15', '15', '14', '13', '12', '12', '12', '11', '11', '11', '11', '10', '10', '10', '10', '10', '09', '09', '09', '09', '09', '09', '09', '08', '08', '08', '08', '08', '07', '07', '07', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '05', '05', '05', '05', '04', '97', '95', '19', '19', '19', '19', '19', '19', '19', '19', '18', '18', '18', '18', '18', '18', '18', '18', '18', '18', '18', '18', '18', '18', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '15', '15', '15', '15', '15', '15', '15', '15', '15', '14', '14', '14', '14', '14', '14', '14', '14', '14', '14', '14', '14', '14', '14', '14', '14', '13', '13', '13', '13', '13', '13

In [17]:
df = pd.Series(years)
counts = dict(df.value_counts())
counts_df=pd.DataFrame(counts.items(),columns=["year","count"])
counts_df.year=pd.to_datetime(counts_df.year, format='%y')


In [18]:
counts_df.sort_values(by=['year'])

,year,count
26,1992-01-01,1
27,1993-01-01,1
22,1994-01-01,5
23,1995-01-01,4
25,1996-01-01,1
18,1997-01-01,9
21,1998-01-01,6
24,1999-01-01,1
20,2000-01-01,9
19,2001-01-01,9


In [19]:
alt.Chart(counts_df).mark_line(point=True).encode(
    x='year',
    y='count'
).properties(
    title='Fallos de la Corte Constitucional de Colombia sobre aborto'
)

alt.Chart(...)

**Challenge:** hacer esta línea de tiempo con todos los fallos
**Challenge:** hacer una línea de tiempo en la que se trace una línea diferente por cada tipo de fallo (Sentencia unificadora,Sentencia de constitucionalidad, Auto o Tutela)

In [20]:
sentencias_df={}
for link in links:
    sentencias_df[link]={}
    try:
        tipo=re.findall(r"[0-9][0-9][0-9][0-9]\/(.*?)\-",link)[0]
        print(tipo)
        #print(tipo)
        tipo=tipo[0]
        year=re.findall(r"[0-9][0-9].htm",link)[0]
        year=year[0:2]
        sentencias_df[link]["tipo"]=tipo
        sentencias_df[link]["year"]=year
        
    except:
        next

sentencias_df=pd.DataFrame.from_dict(sentencias_df,orient='index')
sentencias_df

A574
A558
A333
A140
A616
A557
A547
A642
A641
A448
A060
A531
A330
A281
A181
A502
A486
A009
A117
A098
A244
A096
A038
A273
A172
A161
A085A
A381
A327
A283
A210
A069
A339
A334
A330
A279
A216
A177
A160
A292
A251
A237
A092
A087
A289
A205
A011
A361
A360
A359
A358
A330
A326
A307
A283A
A174
A144
A143
A121
A091
A090
A026
A188A
A159
A149
A119
A172
A011
A041A
T
T
T
T
T
SU062
C
C
C
T
T
T
T
T
T
SU096
SU075
C
C
C
C
C
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
SU677
C
C
C
C
C
T
T
T
T
T
T
T
T
T
T
T
SU214
SU108
C
C
C
C
C
C
C
C
C
C
C
T
T
T
T
T
C
C
C
C
T
T
T
T
T
T
T
T
C
C
C
C
C
C
C
C
T
T
T
T
T
SU071
SU070
C
C
T
T
T
T
T
T
T
T
T
T
T
T
C
C
C
C
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
C
C
C
C
C
C
C
C
T
T
T
T
T
T
T
T
T
C
C
C
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
SU038
C
C
C
C
C
C
C
C
C
C
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
C
C
C
C
C
C
C
C
C
C
T
T
T
T
T
T
T
T
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
A330
T
T
T
T
T
T
T
T
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
T
T
T
T
T
T
T
T
T
T
T
C
T
T
T
T
T
SU383
C
C
C
C
C
C
T


,tipo,year
https://www.corteconstitucional.gov.co/Relatoria/autos/2019/A574-19.htm,A,19
https://www.corteconstitucional.gov.co/Relatoria/autos/2019/A558-19.htm,A,19
https://www.corteconstitucional.gov.co/Relatoria/autos/2019/A333-19.htm,A,19
https://www.corteconstitucional.gov.co/Relatoria/autos/2019/A140-19.htm,A,19
https://www.corteconstitucional.gov.co/Relatoria/autos/2018/A616-18.htm,A,18
...,...,...
https://www.corteconstitucional.gov.co/Relatoria/1994/T-339-94.htm,T,94
https://www.corteconstitucional.gov.co/Relatoria/1994/C-221-94.htm,C,94
https://www.corteconstitucional.gov.co/Relatoria/1994/C-133-94.htm,C,94
https://www.corteconstitucional.gov.co/Relatoria/1993/T-179-93.htm,T,93


In [21]:
counts= sentencias_df.groupby(["year","tipo"]).size()
counts=counts.to_frame(name = 'count').reset_index()
counts.year=pd.to_datetime(counts.year, format='%y')
counts.sort_values(by=['year'])

,year,tipo,count
64,1992-01-01,T,1
65,1993-01-01,T,1
67,1994-01-01,T,3
66,1994-01-01,C,2
70,1995-01-01,T,2
...,...,...,...
56,2018-01-01,A,3
63,2019-01-01,T,5
62,2019-01-01,S,1
61,2019-01-01,C,2


In [22]:
alt.Chart(counts).mark_line(point=True).encode(
    x='year',
    y='count',
    color="tipo"
).properties(
    title='Fallos de la Corte Constitucional de Colombia sobre aborto'
)

alt.Chart(...)